# TMDB 5000 Movie Dataset으로 장르기반 추천시스템 만들기
* 장르에 있는 텍스트를 벡터화 후 텍스트간 유사도를 구해 가까운 순서대로 정렬
* 유사도 척도: 코사인유사도

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv")
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [4]:
data['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
import json

In [10]:
json.loads(data['genres'][0])[0]['name']

'Action'

In [11]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [12]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [13]:
dict_cols = ['genres', 'keywords', 'production_companies','production_countries','spoken_languages']
dict_cols

['genres',
 'keywords',
 'production_companies',
 'production_countries',
 'spoken_languages']

In [30]:
def dict_unpack(data):
    for col in dict_cols:
        temp = []
        for i in json.loads(data[col]):
            temp.append(i.get('name', 'None'))
#        print(temp)
        temp = " ".join(temp)
#        print(temp)
        data[col] = temp
    return data

In [32]:
data = data.apply(dict_unpack, axis=1)

In [33]:
data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners Twentieth Century Fox ...,United States of America United Kingdom,2009-12-10,2787965087,162.0,English Español,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures Jerry Bruckheimer Films S...,United States of America,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures Danjaq B24,United Kingdom United States of America,2015-10-26,880674609,148.0,Français English Español Italiano Deutsch,Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures Warner Bros. DC Entertainme...,United States of America,2012-07-16,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,United States of America,2012-03-07,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,Action Crime Thriller,NaN,9367,united states–mexico barrier legs arms paper k...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,Columbia Pictures,Mexico United States of America,1992-09-04,2040920,81.0,Español,Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,Comedy Romance,NaN,72766,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,,,2011-12-26,0,85.0,,Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,Front Street Pictures Muse Entertainment Enter...,United States of America,2013-10-13,0,120.0,English,Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,,http://shanghaicalling.com/,126186,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,,United States of America China,2012-05-03,0,98.0,English,Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7


In [34]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [36]:
data = data[['original_title','genres','popularity', 'runtime','vote_average','vote_count']]
data

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,139.082615,169.0,6.9,4500
2,Spectre,Action Adventure Crime,107.376788,148.0,6.3,4466
3,The Dark Knight Rises,Action Crime Drama Thriller,112.312950,165.0,7.6,9106
4,John Carter,Action Adventure Science Fiction,43.926995,132.0,6.1,2124
...,...,...,...,...,...,...
4798,El Mariachi,Action Crime Thriller,14.269792,81.0,6.6,238
4799,Newlyweds,Comedy Romance,0.642552,85.0,5.9,5
4800,"Signed, Sealed, Delivered",Comedy Drama Romance TV Movie,1.444476,120.0,7.0,6
4801,Shanghai Calling,,0.857008,98.0,5.7,7


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4803 non-null   object 
 1   genres          4803 non-null   object 
 2   popularity      4803 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4803 non-null   float64
 5   vote_count      4803 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 225.3+ KB


In [39]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4801 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4801 non-null   object 
 1   genres          4801 non-null   object 
 2   popularity      4801 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4801 non-null   float64
 5   vote_count      4801 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 262.6+ KB


# 장르기반 추천시스템 만들기
* 장르 컬럼의 텍스트를 숫자로 벡터화
* 코사인 유사도를 구해 비슷한 장르의 행 검색
* 유사도가 높은 행을 기준으로 내림차순 정렬

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
cv = CountVectorizer(ngram_range=(1, 2))
genres_mat = cv.fit_transform(data['genres'])
print(len(cv.get_feature_names_out()))
genres_mat_df = pd.DataFrame(genres_mat)
genres_mat_df

276


,0
0,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
1,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
2,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
3,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
4,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
...,...
4796,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
4797,"(np.int32(0), np.int32(44))\t1\n (np.int32(..."
4798,"(np.int32(0), np.int32(90))\t1\n (np.int32(..."
4799,


In [44]:
genres_mat_df.loc[0,:]

0      (np.int32(0), np.int32(0))\t1\n  (np.int32(0...
Name: 0, dtype: object

* 코사인 유사도를 이용해 genres_mat의 유사도 산출 후 genres_sim 생성
* 비슷한 장르의 영화를 찾기 위해서 계산
* 코사인 유사도는 주로 문자, 문서의 유사도를 계산하는데 사용 됨

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
genres_sim = cosine_similarity(genres_mat, genres_mat)
print(genres_sim.shape)

(4801, 4801)


In [48]:
data['genres'][:10]

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
3                 Action Crime Drama Thriller
4            Action Adventure Science Fiction
5                    Fantasy Action Adventure
6                            Animation Family
7            Action Adventure Science Fiction
8                    Adventure Fantasy Family
9                    Action Adventure Fantasy
Name: genres, dtype: object

In [47]:
genres_sim_df = pd.DataFrame(genres_sim)
genres_sim_df

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,1.000000,0.596285,0.447214,0.125988,0.755929,0.596285,0.0,0.755929,0.447214,0.745356,...,0.000000,0.000000,0.000000,0.377964,0.000000,0.149071,0.0000,0.000000,0.0,0.0
1,0.596285,1.000000,0.400000,0.169031,0.338062,0.800000,0.0,0.338062,0.600000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0000,0.000000,0.0,0.0
2,0.447214,0.400000,1.000000,0.338062,0.507093,0.600000,0.0,0.507093,0.200000,0.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.0000,0.000000,0.0,0.0
3,0.125988,0.169031,0.338062,1.000000,0.142857,0.169031,0.0,0.142857,0.000000,0.169031,...,0.377964,0.169031,0.377964,0.428571,0.218218,0.676123,0.0000,0.125988,0.0,0.0
4,0.755929,0.338062,0.507093,0.142857,1.000000,0.507093,0.0,1.000000,0.169031,0.507093,...,0.000000,0.000000,0.000000,0.428571,0.000000,0.169031,0.0000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4796,0.149071,0.200000,0.400000,0.676123,0.169031,0.200000,0.0,0.169031,0.000000,0.200000,...,0.000000,0.200000,0.000000,0.169031,0.258199,1.000000,0.0000,0.000000,0.0,0.0
4797,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.258199,0.000000,0.000000,0.000000,0.000000,1.0000,0.384900,0.0,0.0
4798,0.000000,0.000000,0.000000,0.125988,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.333333,0.149071,0.333333,0.125988,0.000000,0.000000,0.3849,1.000000,0.0,0.0
4799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0


In [50]:
sorted_genres_sim = genres_sim.argsort()[:, ::-1]

In [51]:
sorted_genres_sim[:1]

array([[  14,  870,   46, ..., 4724, 4723, 3572]])

In [52]:
data.loc[0,:]

original_title                                      Avatar
genres            Action Adventure Fantasy Science Fiction
popularity                                      150.437577
runtime                                              162.0
vote_average                                           7.2
vote_count                                           11800
Name: 0, dtype: object

In [53]:
data.loc[14,:]

original_title                                Man of Steel
genres            Action Adventure Fantasy Science Fiction
popularity                                       99.398009
runtime                                              143.0
vote_average                                           6.5
vote_count                                            6359
Name: 14, dtype: object

# 영화 이름과 추천 받을 개수를 입력받아 추천영화 출력하기

In [58]:
movie_index = data[data['original_title'] == 'Avatar'].index.values[0]

In [62]:
recommaned_idx = sorted_genres_sim[movie_index, 1:11]

In [63]:
data.loc[recommaned_idx, :]

,original_title,genres,popularity,runtime,vote_average,vote_count
870,Superman II,Action Adventure Fantasy Science Fiction,30.515175,127.0,6.5,629
46,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,118.078691,131.0,7.5,6032
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800
3493,Morvern Callar,Drama,2.507912,97.0,7.2,34
813,Superman,Action Adventure Fantasy Science Fiction,48.507081,143.0,6.9,1022
1652,Dragonball Evolution,Action Adventure Fantasy Science Fiction Thriller,21.677732,85.0,2.9,462
1296,Superman III,Comedy Action Adventure Fantasy Science Fiction,22.164202,125.0,5.3,490
419,Jumper,Adventure Fantasy Science Fiction,21.218000,88.0,5.9,1799
420,Hellboy II: The Golden Army,Adventure Fantasy Science Fiction,58.579760,120.0,6.5,1527
3207,Awake,Thriller Crime Mystery,18.172736,84.0,6.3,395


In [64]:
def find_sim_movie(data, sorted_genres_sim, title_name, top_n=10):
    title_movie = data[data['original_title'] == title_name]
    title_index = title_movie.index.values[0]
    similer_idxs = sorted_genres_sim[title_index, 1:top_n+1]
    return data.iloc[similer_idxs]

In [66]:
find_sim_movie(data, sorted_genres_sim, "Superman III", 19)

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800
3494,Beastmaster 2: Through the Portal of Time,Action Adventure Fantasy Science Fiction,1.478505,107.0,4.6,17
870,Superman II,Action Adventure Fantasy Science Fiction,30.515175,127.0,6.5,629
46,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,118.078691,131.0,7.5,6032
14,Man of Steel,Action Adventure Fantasy Science Fiction,99.398009,143.0,6.5,6359
813,Superman,Action Adventure Fantasy Science Fiction,48.507081,143.0,6.9,1022
1191,Small Soldiers,Comedy Adventure Fantasy Science Fiction Action,23.088571,110.0,6.2,511
1932,Sheena,Action Adventure Comedy Fantasy Science Fiction,4.020194,117.0,5.0,22
238,Teenage Mutant Ninja Turtles,Science Fiction Action Adventure Fantasy Comedy,143.350376,101.0,5.8,2636
1652,Dragonball Evolution,Action Adventure Fantasy Science Fiction Thriller,21.677732,85.0,2.9,462
